In [1]:
!pip install confluent_kafka

**Note**: Use cd .. or pwd commands to read the data files if required.

In [1]:
cd ..

e:\ASTUDY\HK-5\BIGDATA\IE212.O11.Group7


C:\Users\NUNNUN\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\IPython\core\magics\osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
import json
import gzip
from confluent_kafka import Producer
import time

In [3]:
def read_reviews(file_path):
    with gzip.open(file_path) as file:
        for line_number, line in enumerate(file, start=100000):
            # review = json.loads(eval(line))
            review = eval(line)
            yield review

def kafka_producer(broker, topic, file_path):
    producer_conf = {'bootstrap.servers': broker}
    producer = Producer(producer_conf)
    x = 0

    try:
        for review in read_reviews(file_path):
            x += 1
            
            review['last_update_time'] = time.time()
            # Convert the review to a JSON string
            message_value = json.dumps(review)
            
            # Produce the message to the Kafka topic
            producer.produce(topic, value=message_value.encode('utf-8'))

            # Flush the producer to ensure the message is sent
            producer.flush()

            # Print the data just sent
            print(f"Sent {x} :  {message_value}")

            # Pause for 5 seconds
            time.sleep(5)

    except KeyboardInterrupt:
        pass

    finally:
        # Close the producer (no close method, just flush)
        producer.flush()

In [4]:
if __name__ == '__main__':
    # Thay thế 'localhost:9092' và 'PREDICT_RATINGS_OF_GOOGLE_LOCAL_REVIEWS_IE212_O11_GROUP7' bằng thông tin Kafka của bạn
    bootstrap_servers = 'localhost:9092'
    kafka_topic = 'PREDICT_RATINGS_OF_GOOGLE_LOCAL_REVIEWS_IE212_O11_GROUP7'
    
    # Thay thế 'data/reviews.clean.json.gz' bằng đường dẫn tới tệp của bạn
    file_path = 'data/reviews_TX.json.gz'
    
    try:
        kafka_producer(bootstrap_servers, kafka_topic, file_path)
    except KeyboardInterrupt:
        print("Stopping the producer...")
        running = False  # Đặt biến cờ thành False để dừng producer

Sent 1 :  {"rating": 5.0, "reviewerName": "Johanna S", "reviewText": "My husband uses his truck for work, and several months ago we started having problems with it. We needed it fixed ASAP in order for him to get back to work. It was a Thursday and I called the closest dealership, Lithia in Corpus, and they said they wouldn't be able to even look at it until the following Wednesday. So I called Port Lavaca Dodge. They told me to bring it that day, and they would do their best to get it looked at.  Long story short, the truck was up and running good as new by the following Wednesday. That's why we continue to drive the extra hour to Port Lavaca to have both our trucks serviced there.", "categories": ["Car Dealer"], "gPlusPlaceId": "104231480695579463827", "unixReviewTime": 1362944666, "reviewTime": "Mar 10, 2013", "gPlusUserId": "100000050936341836015", "last_update_time": 1706053234.648046}


Sent 2 :  {"rating": 5.0, "reviewerName": "Jason Wagner", "reviewText": "I don't live in Austin (I live in Cali), but I go there regularly for work.  And every time I'm there, I always make a stop by Bobalu's for some cigars to bring home.  Why?  - The cigars are fresh rolled there in the store.  Sometimes I get to watch and choose the ones that were just rolled. - The cigars are tasty and there is a wide variety of sizes and flavors. - The service is great.  Walk into the large humidor and someone will always come by to answer any questions without being pushy.  Very relaxed atmosphere. - Oh yeah, and the prices are great.  Don't know what you want?  They have variety boxes at a great price (normally what I get).  Why go pay way more for an imported cigar from a place where it may have been sitting there for a long time?  You can get a warm, fuzzy feeling knowing that you are helping out a local small business.  Go check 'em out, you won't be disappointed!!!", "categories": ["Cigar Sh